In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from matplotlib import font_manager, rc
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import nltk
import chardet

%matplotlib inline

# nltk 데이터 다운로드 (첫 실행 시 필요)
nltk.download('vader_lexicon')

# nltk 데이터 다운로드 (첫 실행 시 필요)
nltk.download('stopwords')

# 글꼴 경로 지정
font_path = "c:/Windows/Fonts/malgun.ttf"  # 윈도우에 설치된 맑은 고딕 폰트 경로

# 폰트 이름 얻어오기
font_name = font_manager.FontProperties(fname=font_path).get_name()

# matplotlib의 rc(run command) 기능을 이용하여 글꼴 설정
mpl.rc('font', family=font_name)

# 유니코드에서  음수 부호 설정
mpl.rc('axes', unicode_minus=False)

raw = pd.read_csv('../../../../../datasets/paris_reviews_check1.csv')
df = raw.copy()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ciw96\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ciw96\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
len(raw[raw['is_english']==False])

0

In [9]:
df.head()

,숙소_id,리뷰날짜,리뷰,vader_sentiment,textblob_sentiment,부정,중립,긍정,전체,is_english
0,2104349,2019-03-03,Strategically positioned apartment (in two lev...,"{'neg': 0.01, 'neu': 0.842, 'pos': 0.148, 'com...",0.307619,0.01,0.842,0.148,0.9701,True
1,904241734383540468,2023-09-09,Beautiful place. one bedroom with a bedsofa in...,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...",0.375040,0.00,0.769,0.231,0.6908,True
2,23476199,2018-05-29,Alexandra's apartment is like a trip back in t...,"{'neg': 0.0, 'neu': 0.69, 'pos': 0.31, 'compou...",0.277292,0.00,0.690,0.310,0.9622,True
3,723879972115619669,2022-12-02,"Wonderful apartment, felt right at home. Love...","{'neg': 0.0, 'neu': 0.571, 'pos': 0.429, 'comp...",0.471429,0.00,0.571,0.429,0.8176,True
4,25025384,2022-04-29,"Good place, close to train station! Alex is ve...","{'neg': 0.0, 'neu': 0.494, 'pos': 0.506, 'comp...",0.655938,0.00,0.494,0.506,0.9609,True


In [12]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize

# nltk 토크나이저 로드
import nltk
nltk.download('punkt')

# 데이터 로딩
df = pd.read_csv('../../../../../datasets/paris_reviews_check1.csv')
df = df[['숙소_id', '리뷰날짜', '리뷰']]

# 전처리: 영어와 공백만 남김
df['review'] = df['리뷰'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', str(x)))

# 토큰화 함수 정의
def english_tokenizer(text):
    return word_tokenize(text.lower())

# TF-IDF 벡터화
tfidf = TfidfVectorizer(tokenizer=english_tokenizer, ngram_range=(1, 2), min_df=3, max_df=0.9)
tfidf_matrix = tfidf.fit_transform(df['review'])



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ciw96\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [13]:
tfidf_matrix

<18591x65123 sparse matrix of type '<class 'numpy.float64'>'
	with 1947133 stored elements in Compressed Sparse Row format>

In [2]:
df.head()

,숙소_id,리뷰날짜,리뷰
0,2104349,2019-03-03,Strategically positioned apartment (in two lev...
1,904241734383540468,2023-09-09,Beautiful place. one bedroom with a bedsofa in...
2,23476199,2018-05-29,Alexandra's apartment is like a trip back in t...
3,723879972115619669,2022-12-02,"Wonderful apartment, felt right at home. Love..."
4,25025384,2022-04-29,"Good place, close to train station! Alex is ve..."


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

# nltk 데이터 다운로드 (첫 실행 시 필요)
nltk.download('stopwords')
print(type(stopwords.words('english')))
# 긍정적 리뷰와 부정적 리뷰로 분류
positive_reviews = df[df['전체'] >= 0.05]['리뷰'].dropna().astype(str)
negative_reviews = df[df['전체'] <= -0.05]['리뷰'].dropna().astype(str)

# 영어, 스페인어, 독일어 불용어 목록 결합 및 리스트로 변환
stop_words = list(set(stopwords.words('english')) | set(stopwords.words('spanish')) | set(stopwords.words('german')))

# 추가로 제외할 단어 목록
additional_stop_words =  ['paris','super','perfect','place', 'stay', 'airbnb', 'would', 'us','great','nice','good','amazing','highly']

# 추가적인 불용어 목록을 결합
stop_words.extend(additional_stop_words)

# TF-IDF 벡터라이저 초기화 (결합된 불용어 목록 사용)
tfidf_vectorizer = TfidfVectorizer(max_features=100, stop_words=stop_words)

# 긍정적 리뷰에 대한 TF-IDF 행렬 계산
tfidf_matrix_positive = tfidf_vectorizer.fit_transform(positive_reviews)
feature_names_positive = tfidf_vectorizer.get_feature_names_out()
tfidf_scores_positive = tfidf_matrix_positive.sum(axis=0).A1
tfidf_df_positive = pd.DataFrame({'word': feature_names_positive, 'tfidf_score': tfidf_scores_positive})
top_tfidf_positive = tfidf_df_positive.nlargest(20, 'tfidf_score')  # 상위 20개 단어

# 부정적 리뷰에 대한 TF-IDF 행렬 계산 (불용어 목록 조정)
try:
    tfidf_matrix_negative = tfidf_vectorizer.fit_transform(negative_reviews)
    feature_names_negative = tfidf_vectorizer.get_feature_names_out()
    tfidf_scores_negative = tfidf_matrix_negative.sum(axis=0).A1
    tfidf_df_negative = pd.DataFrame({'word': feature_names_negative, 'tfidf_score': tfidf_scores_negative})
    top_tfidf_negative = tfidf_df_negative.nlargest(20, 'tfidf_score')  # 상위 20개 단어

    # 결과 출력
    print("긍정적인 리뷰에서 상위 20개 단어:")
    print(top_tfidf_positive)
except ValueError as e:
    print(f"TF-IDF 계산 중 오류 발생: {e}")
    print("부정적 리뷰의 예시:")
    print(negative_reviews.head(10))  # 부정적 리뷰 예시 출력


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ciw96\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<class 'list'>


NameError: name 'df' is not defined

In [33]:
print("\n부정적인 리뷰에서 상위 20개 단어:")
print(top_tfidf_negative)


부정적인 리뷰에서 상위 20개 단어:
         word  tfidf_score
1   apartment   925.126654
52   location   632.267834
37       host   578.785336
74       room   474.600973
64        one   432.086472
11        bed   414.898306
16      check   409.445395
78      small   392.384281
17      clean   378.955139
61      night   367.834685
10   bathroom   360.023696
76     shower   345.391763
87       time   334.820329
34        get   333.058021
24      dirty   331.416706
72     really   314.275127
32      floor   302.532662
25       door   297.347049
31       flat   293.504640
56      metro   292.095416


In [34]:
!pip install --upgrade scipy gensim

In [35]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
import networkx as nx
from nltk.corpus import stopwords
import nltk

# 1. Term Frequency (단어 빈도)
def term_frequency(reviews, stop_words):
    words = [word for word in ' '.join(reviews).split() if word.lower() not in stop_words]
    word_counts = Counter(words)
    return [word for word, _ in word_counts.most_common(20)]

most_common_positive = term_frequency(positive_reviews, stop_words)
most_common_negative = term_frequency(negative_reviews, stop_words)

# 2. Latent Dirichlet Allocation (LDA)
def lda_keywords(reviews, stop_words):
    count_vectorizer = CountVectorizer(stop_words=stop_words)
    count_data = count_vectorizer.fit_transform(reviews)
    lda = LatentDirichletAllocation(n_components=1, random_state=42)
    lda.fit(count_data)
    topic = lda.components_
    feature_names = count_vectorizer.get_feature_names_out()
    top_words = [feature_names[i] for i in topic[0].argsort()[-20:]]
    return top_words

top_words_positive_lda = lda_keywords(positive_reviews, stop_words)
top_words_negative_lda = lda_keywords(negative_reviews, stop_words)

# 3. TextRank
def textrank_keywords(texts, stop_words, top_n=20):
    vectorizer = TfidfVectorizer(stop_words=stop_words)
    X = vectorizer.fit_transform(texts)
    feature_names = vectorizer.get_feature_names_out()
    
    # 단어 간의 유사성 그래프 생성
    similarity_graph = (X.T * X)
    similarity_graph.setdiag(0)
    similarity_graph = similarity_graph.toarray()
    
    # 네트워크 그래프 생성
    nx_graph = nx.from_numpy_array(similarity_graph)
    scores = nx.pagerank(nx_graph)
    
    # 점수 기준 상위 단어 추출
    ranked_words = sorted(((scores[i], s) for i, s in enumerate(feature_names)), reverse=True)
    top_keywords = [word for _, word in ranked_words[:top_n]]
    
    return top_keywords

positive_keywords_textrank = textrank_keywords(positive_reviews, stop_words)
negative_keywords_textrank = textrank_keywords(negative_reviews, stop_words)

# 4. TF-IDF
def tfidf_keywords(reviews, stop_words, top_n=20):
    vectorizer = TfidfVectorizer(stop_words=stop_words)
    X = vectorizer.fit_transform(reviews)
    feature_names = vectorizer.get_feature_names_out()
    scores = X.sum(axis=0).A1
    sorted_indices = scores.argsort()[::-1]
    top_keywords = [feature_names[i] for i in sorted_indices[:top_n]]
    return top_keywords

top_words_positive_tfidf = tfidf_keywords(positive_reviews, stop_words)
top_words_negative_tfidf = tfidf_keywords(negative_reviews, stop_words)

# 겹치는 단어 찾기
def find_common_words(lists):
    common_words = set(lists[0])
    for lst in lists[1:]:
        common_words.intersection_update(lst)
    return list(common_words)

positive_lists = [most_common_positive, top_words_positive_lda, positive_keywords_textrank, top_words_positive_tfidf]
negative_lists = [most_common_negative, top_words_negative_lda, negative_keywords_textrank, top_words_negative_tfidf]

common_positive_words = find_common_words(positive_lists)
common_negative_words = find_common_words(negative_lists)

print("긍정적인 리뷰에서 공통적으로 상위에 있는 단어:")
print(common_positive_words)

print("\n부정적인 리뷰에서 공통적으로 상위에 있는 단어:")
print(common_negative_words)

긍정적인 리뷰에서 공통적으로 상위에 있는 단어:
['host', 'easy', 'close', 'apartment', 'metro', 'restaurants', 'recommend', 'well', 'everything', 'location', 'really', 'lovely', 'comfortable', 'clean']

부정적인 리뷰에서 공통적으로 상위에 있는 단어:
['host', 'check', 'door', 'apartment', 'location', 'shower', 'room', 'night', 'bathroom', 'small', 'time', 'get', 'bed', 'one']


In [24]:
top_words_positive_lda

['definitely',
 'flat',
 'check',
 'helpful',
 'time',
 'walk',
 'lovely',
 'easy',
 'really',
 'restaurants',
 'comfortable',
 'close',
 'well',
 'recommend',
 'everything',
 'metro',
 'clean',
 'host',
 'location',
 'apartment']

In [25]:
top_words_negative_lda

['dirty',
 'metro',
 'could',
 'people',
 'clean',
 'door',
 'floor',
 'time',
 'small',
 'check',
 'two',
 'get',
 'bathroom',
 'bed',
 'night',
 'room',
 'one',
 'location',
 'host',
 'apartment']

In [26]:
positive_reviews

0        Great location and great places to eat nearby....
1        It is a lovely small apartment, very quite and...
3        Wonderful apartment in a central, exciting nei...
4        One of the best airbnbs we have ever stayed in...
5        Eleonor is a really great, responsive host.  T...
                               ...                        
19995                           Great stay, great location
19996    We spent a lot of time exploring Paris and aft...
19997    Apartment was very nice and comfortable. All w...
19998    Very cute well designed use of a small space. ...
19999    Very nice apartment in a comparatively quiet n...
Name: 리뷰, Length: 19517, dtype: object

In [27]:
negative_reviews

2        The appartement was fine, close to the metro s...
156      I had a horrible experience at this place. I r...
169      Vincent strikes me as someone who is tired of ...
479      One key for 8 people. Poor 4 rolls of tp for 8...
481      Good location.  Good size apartment, however w...
                               ...                        
19823    The apartment is unique and in a great locatio...
19831        good place but the sound proof is not perfect
19845    Everything was great. Clean and nice little pl...
19886    It is a dirty place, I think they never clean ...
19923    Love the hotel (amazing value for money) but h...
Name: 리뷰, Length: 305, dtype: object

NameError: name 'corpora' is not defined